# Problema: Cuentas Bancarias

Una institución bancaria dispone de una base de datos que utiliza para el control de las cuentas corrientes de sus clientes. Esta base de datos se compone de tres archivos. 

El archivo `clientes.txt` registra el RUT y el nombre de los clientes, utilizando como separador de campos el símbolo punto y coma (`;`). Por ejemplo,

```
12367266K;Alexis Ford
78926919;Danny Elfman 
141117641;Rebecca Ferguson
97662918;Nathan Fillion
.
:
```

El archivo `cuentas.txt` registra las cuentas de los clientes. La asociación cliente-cuenta se realiza a partir del RUT del cliente, el número de la cuenta y el estado de esta última. Las cuentas pueden estar en uno de dos estados: activa (`1`) o inactiva (`0`). Si una cuenta está inactiva, no se pueden realizar transacciones desde y hacie esta. Además, un cliente puede tener una o más cuentas, es decir, un registro del archivo `clientes.txt` puede estar asociado a uno o más registros del archivo `cuentas.txt`. Por ejemplo, el archivo `clientes.txt` puede contener los siguientes registros:

```
12367266K:1234-5678-8765-4321:1
97662918:0001-3456-2223-7676:1
78926919:4321-6548-0015-4001:1
12367266K:2231-0021-0400-1101:0
.
:
```

El archivo `movimientos.txt` mantiene un registro de las transacciones de dinero en las cuentas. Por cada transacción se registra la fecha, la cuenta, el monto y el tipo de transacción que se realiza sobre una cuenta. El campo tipo registra una de las siguientes transacción posibles: giro por caja (`GC`), giro por cajero automático (`GA`) y depósito (`DP`). Los giros sólo pueden ser realizados desde cuentas activas. Cuando los giros se realizan por caja (`GC`), no existe límite del monto. Cuando se realizan por cajero automático (`GA`), el monto no puede superar los &dollar;200.000 por día, ya sea por transacción o de forma acumulada.

El siguiente corresponde a un ejemplo del archivo `movimientos.txt`.

```
2020-01-12 1234-5678-8765-4321 25000 GA
2020-02-01 4321-6548-0015-4001 120000 DP
.
:
```

Se requiere diseñar un archivo que contenga las siguientes funciones.

A. La función `transaccion(cuenta, tipo, monto, fecha)` registra un movimiento de dinero en una cuenta, a partir del número de `cuenta`, el `tipo` de transacción y el `monto` de la transacción. Si la transacción es válida, esta deberá ser registrada en el archivo `movimientos.txt` y la función deberá devolver el valor `True`. En los casos que la transacción no sea válida, la función deberá retornar el valor `False`.

A continuación se muestran algunos ejemplos de la ejecución de la función:

```
>>> transaccion('4321-6548-0015-4001', 'GC', 10000, '2021-12-13')
La cuenta no existe o está desactiva.
False

>>> transaccion('1321-6548-0015-4001', 'GA', 500000, '2021-03-12')
ATM permite girar hasta 200 000 al día.
False

>>> transaccion('1234-5678-8765-4321', 'GA', 175000, '2020-01-12')
True

>>> transaccion('1234-5678-8765-4321', 'GA', 25001, '2020-01-12')
ATM permite girar hasta 200 000 al día.
False
```

Análisis:

1. Se requiere diseñar la función `obtener_cuenta(numero) -> (rut, cuenta, estado)` que encuentre en el archivo `cuentas.txt` el número de cuenta pasado como argumento y retorne una tupla con los datos de la cuenta.
2. La función `transaccion(cuenta, tipo, monto, fecha)` invocará la función `obtener_cuenta()`, si el retorno es una lista vacía, deberá mostrar el mensaje `la cuenta no existe o está desactivada` y retornará el valor `False`. En el caso contrario, deberá validar el tipo de transacción (condicional). 
    - Si la transacción es de tipo depósito (`DP`), se deberá __registrar el movimiento__.
    - Si la transacción es de tipo `GC` o `GA`, se deberá validar el __estado de la cuenta__. 
        - Si el estado de la cuenta es `1`, se deberá verificar que la cuenta tenga saldo suficiente.
            - Si existe saldo suficiente, se deberá validar el tipo de giro.
                - Si es de tipo `GC`, se __registrará el movimiento__. 
                - Si es de tipo `GA` se deberá __obtener el monto (sumatoria) de todos los giros para el día__. A este monto se le deberá sumar el monto que se quiere girar.
                    - Si el monto calculado es mayor a `200 000`, se deberá mostrar el mensaje `ATM permite girar hasta 200 000 al día.`.
                    - En caso contrario, se __registrará el movimiento__.
            - En caso contrario se mostrará el mensaje `Saldo insufuciente en la cuenta`.
            
3. Se requiere diseñar la función `registrar_movimiento(cuenta, fecha, monto, tipo)` que registra los movimientos en el archivo `movimientos.txt`.
4. Se requiere diseñar la función, `saldos(numero_cuenta, fecha_atm) -> (int, int)` que calcula el saldo total de la cuenta, y el saldo de los giros por ATM para la fecha pasada como argumento (opcional). Retornará el salto total y el saldo por giros ATM.

B. La función `saldos(rut)` devuelve un diccionario con los saldos de cada una de las cuentas asociadas al RUT de un cliente (`rut`). Cada clave de este diccionario es el número de una cuenta del cliente y su valor es el saldo asociado a dicha cuenta.

```
>>> saldos('12367266K')
{'1234-5678-8765-4321': 24999, '2231-0021-0400-1101': 0}
```

C. La función `cartola(cuenta, fecha_1, fecha_2)` crea un archivo con el nombre de la cuenta, el cual contiene los movimientos (transacciones) realizados en una cuenta entre dos fechas. Cada línea del archivo creado contiene un número único (que inicia en `1`), la fecha, el monto y el tipo de transacción. Además, la ejecución de la función debe retornar el saldo de la cuenta. 

Por ejemplo, la ejecución de la siguiente instrucción,

```
>>> cartola('1234-5678-8765-4321', '2020-01-01', '2020-01-30')
230000
```

Genera el archivo `1234-5678-8765-4321.txt` con el siguiente formato y contenido:

```
1 2020-01-01 880000 DP
2 2020-01-12 20000 GA
3 2020-01-12 130000 GA
4 2020-01-15 550000 GC
5 2020-01-25 150000 DP
6 2020-01-26 100000 GA
```